In [1]:
!pip install keras-tuner
!pip install tensorflow

     |████████████████████████████████| 133 kB 5.1 MB/s 
     |████████████████████████████████| 462 kB 5.2 MB/s 


In [2]:
import pandas as pd
from tensorflow import keras
from keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [3]:
df = pd.read_csv("/content/post_one_hot_encode.csv")

train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

x_train=train.loc[:, ['Power', 'Fin Type_Round', 'Fin Type_Square', 'Fin Type_Triangular', 'Configuration_Inline', 'Configuration_Staggered', 'PCM_Paraffin', 'PCM_RT-35', 'PCM_RT-44', 'Temperature']]
#x_train=x_train.to_numpy()
y_train=train.loc[:, 'Time']
#y_train=y_train.to_numpy()
x_test=test.loc[:, ['Power', 'Fin Type_Round', 'Fin Type_Square', 'Fin Type_Triangular', 'Configuration_Inline', 'Configuration_Staggered', 'PCM_Paraffin', 'PCM_RT-35', 'PCM_RT-44', 'Temperature']]
#x_test=x_test.to_numpy()
y_test=test.loc[:, 'Time']
#y_test=y_test.to_numpy()

In [4]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_fc_layers', 2, 20)):
    model.add(layers.Dense(units=hp.Int('units_' + str(i), 
                                        min_value=5, 
                                        max_value=50, 
                                        step=5), 
                            activation='relu'))
  model.add(layers.Dense(1, activation='linear'))
  model.compile(
      optimizer='adam',
      loss='mean_absolute_error',
      metrics=['mean_absolute_error'])
  return model

In [5]:
tuner = RandomSearch(
    build_model,
    objective = 'val_mean_absolute_error',
    max_trials = 100,
    executions_per_trial = 3,
    directory="project3",
    project_name="helloworld",
)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_fc_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 50, 'step': 5, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 50, 'step': 5, 'sampling': None}


In [7]:
tuner.search(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Trial 100 Complete [00h 00m 07s]
val_mean_absolute_error: 16.670807520548504

Best val_mean_absolute_error So Far: 13.88422934214274
Total elapsed time: 00h 10m 05s
INFO:tensorflow:Oracle triggered exit


In [8]:
#Taking the hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [9]:
#Finding optimal no of epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=500, validation_split=0.2)

val_mean_absolute_error_per_epoch = history.history['val_mean_absolute_error']
best_epoch = val_mean_absolute_error_per_epoch.index(min(val_mean_absolute_error_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/500
34/34 [==============================] - 1s 8ms/step - loss: 39.3325 - mean_absolute_error: 39.3325 - val_loss: 18.7584 - val_mean_absolute_error: 18.7584
Epoch 2/500
34/34 [==============================] - 0s 3ms/step - loss: 18.6068 - mean_absolute_error: 18.6068 - val_loss: 19.0658 - val_mean_absolute_error: 19.0658
Epoch 3/500
34/34 [==============================] - 0s 4ms/step - loss: 17.4347 - mean_absolute_error: 17.4347 - val_loss: 18.1191 - val_mean_absolute_error: 18.1191
Epoch 4/500
34/34 [==============================] - 0s 4ms/step - loss: 16.9815 - mean_absolute_error: 16.9815 - val_loss: 17.4591 - val_mean_absolute_error: 17.4591
Epoch 5/500
34/34 [==============================] - 0s 3ms/step - loss: 16.5567 - mean_absolute_error: 16.5567 - val_loss: 17.1675 - val_mean_absolute_error: 17.1675
Epoch 6/500
34/34 [==============================] - 0s 3ms/step - loss: 16.2642 - mean_absolute_error: 16.2642 - val_loss: 16.5872 - val_mean_absolute_error: 16.587

In [10]:
#Print parameters of the best model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 40)                440       
                                                                 
 dense_16 (Dense)            (None, 50)                2050      
                                                                 
 dense_17 (Dense)            (None, 50)                2550      
                                                                 
 dense_18 (Dense)            (None, 35)                1785      
                                                                 
 dense_19 (Dense)            (None, 30)                1080      
                                                                 
 dense_20 (Dense)            (None, 45)                1395      
                                                                 
 dense_21 (Dense)            (None, 15)               

In [11]:
tuner.results_summary()

Results summary
Results in project3/helloworld
Showing 10 best trials
Trial summary
Hyperparameters:
num_fc_layers: 8
units_0: 40
units_1: 50
units_2: 50
units_3: 35
units_4: 30
units_5: 45
units_6: 15
units_7: 35
units_8: 10
units_9: 5
units_10: 20
units_11: 10
units_12: 15
units_13: 15
units_14: 50
units_15: 30
units_16: 20
units_17: 25
units_18: 15
units_19: 25
Score: 13.88422934214274
Trial summary
Hyperparameters:
num_fc_layers: 18
units_0: 20
units_1: 25
units_2: 25
units_3: 40
units_4: 30
units_5: 35
units_6: 15
units_7: 20
units_8: 15
units_9: 15
units_10: 10
units_11: 40
units_12: 5
units_13: 25
units_14: 25
units_15: 40
units_16: 25
units_17: 20
Score: 14.295278867085775
Trial summary
Hyperparameters:
num_fc_layers: 18
units_0: 35
units_1: 5
units_2: 30
units_3: 30
units_4: 20
units_5: 5
units_6: 10
units_7: 40
units_8: 40
units_9: 15
units_10: 20
units_11: 10
units_12: 35
units_13: 45
units_14: 30
units_15: 20
units_16: 50
units_17: 45
units_18: 30
units_19: 5
Score: 14.3337